In [ ]:
import numpy as np
import pandas as pd
import yaml
import json
import os
import sys
from dotenv import load_dotenv

load_dotenv()
REPO_PATH = os.getenv("REPO_PATH")
sys.path.insert(0, rf'{REPO_PATH}src_HF')

from utils.forecast_utils import RNNGenerator
from utils.model_utils import optimize_hyperparameters

### Import data

In [ ]:
# load yaml file
with open('variable_config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

# Define feature list and target
feature_columns = list()

feature_columns += config['BASE']
feature_columns += config['SENTIMENT_CUSTOM']
feature_columns += config['TEMPORAL']

# Specify RNN model. Alternatives: BiLSTM, GRU or BiGRU
rnn_type='LSTM'

# Specify data
future = 'CLc1'
rnn_type='LSTM'

data_params = {
    'feature_columns': feature_columns,
    'target_column': 'TARGET_1',
    'window_size': 30,
    'test_size': 0.2,
    'val_size': 0.2,
    'CV': False
}

# Specify data
gen = RNNGenerator(future='CLc1', CV=data_params['CV'])

gen.preprocess_data(
    data_params['feature_columns'], 
    data_params['target_column'], 
    data_params['window_size'],
    test_size=data_params['test_size'],
    val_size=data_params['val_size']
)


In [ ]:
trial_params = {
        'units_first_layer': [16, 32, 64, 128],
        'units_second_layer': [16, 32, 64, 96],
        'dropout_rate_first': [0.1, 0.5],
        'dropout_rate_second': [0.1, 0.5],
        'l2_strength': [1e-5, 1e-3],
        'learning_rate': [1e-5, 1e-2],
        'batch_size': [16, 32, 64],
        'noise_std': [0.01, 0.1]
}

# Find the best hyperparameters using Optuna
best_params = optimize_hyperparameters(
    gen.train_generators[-1],
    gen.val_generators[-1],
    trial_params,
    data_params['target_column'], 
    rnn_type,
    data_params['window_size'], 
    n_trials=50, 
    n_jobs=-1
)

current_dt = pd.Timestamp.now().strftime('%Y.%m.%d_%H.%M')
hp_filename = f'{future}_{rnn_type}_{current_dt}'

print(f'Best parameters for {rnn_type} model: ', best_params)

# Save best hyperparameters to .json file
with open(f'hyperpm_archive/{hp_filename}.json', 'w') as file:
    json.dump(best_params, file, indent=4)
